### Use case notebook for the cadence python package ###

In [1]:
import os
import glob
import random
import numpy as np

# There is a warning in the timeboard library that we want to suppress here
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
from pathlib import Path
import time
import timeboard as tb
import timeboard.calendars.US as US
import datetime
import holidays
from faker import Faker
import operator
from functools import reduce
from itertools import chain

# Appearance of the Notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Import the package
%load_ext autoreload
%autoreload 2
import cadence
from cadence.mscheduler import Meetings
from cadence.utils import GroupFaker
print(f'Package version: {cadence.__version__}')

Package version: 0.0.2.post1.dev9+g55f2059.d20240827


### Helper functions ###

### Create the data: meeting participants and research groups ###

In [2]:
n_members = 10
n_groups = 3
gf = GroupFaker(n_members=n_members, n_groups=3)

name_df = gf.create_fake_research_group()
display(name_df)

,name,first_name,last_name,group
0,Brandon Russell,Brandon,Russell,group_0
1,Jerome Whitehead,Jerome,Whitehead,group_0
2,Ryan Mack,Ryan,Mack,group_0
3,Thomas Berry,Thomas,Berry,group_0
4,Charles Tyler,Charles,Tyler,group_1
5,Evelyn Christian,Evelyn,Christian,group_1
6,Steven Johnson,Steven,Johnson,group_1
7,Aaron Graham,Aaron,Graham,group_2
8,George Cook,George,Cook,group_2
9,Kyle Jones,Kyle,Jones,group_2


### Instantiate the Meeting class ###

In [3]:
groups = list(name_df['group'].values)
names = list(name_df['first_name'].values)
meet = Meetings(name_list=names, group_list=groups)
display(meet.name_df)

,name,group
0,Brandon,group_0
1,Jerome,group_0
2,Ryan,group_0
3,Thomas,group_0
4,Charles,group_1
5,Evelyn,group_1
6,Steven,group_1
7,Aaron,group_2
8,George,group_2
9,Kyle,group_2


### Create a presenter list with a random sequence of names from alternating groups ###
This is because we may want a member of a different group present every week

In [4]:
# Instantiate the Meetings instance with the original data
groups = list(name_df['group'].values)
names = list(name_df['name'].values)
meet = Meetings(name_list=names, group_list=groups)

# Create a presenter list with the names from alternating groups
presenter_list = meet.create_name_sequence()
print(presenter_list)
print()
# This changes the sequence of names in the data frame of the instance
display(meet.name_df)

['Brandon Russell', 'Charles Tyler', 'Aaron Graham', 'Jerome Whitehead', 'Evelyn Christian', 'George Cook', 'Ryan Mack', 'Steven Johnson', 'Kyle Jones', 'Thomas Berry']



,name,group
0,Brandon Russell,group_0
1,Charles Tyler,group_1
2,Aaron Graham,group_2
3,Jerome Whitehead,group_0
4,Evelyn Christian,group_1
5,George Cook,group_2
6,Ryan Mack,group_0
7,Steven Johnson,group_1
8,Kyle Jones,group_2
9,Thomas Berry,group_0


### Manipulate the sequence of presenters ###

In [5]:
# Instantiate the Meetings instance with the original data
groups = list(name_df['group'].values)
names = list(name_df['name'].values)
meet = Meetings(name_list=names, group_list=groups)

# This is the list of names that we generated
print(f'Input list of names:\n{names}')

# Create an initial list of presenters from alternating groups
presenter_list_0 = meet.create_name_sequence()
print()
print(f'Presenter list after merging groups:\n{presenter_list_0}')
display(meet.name_df)

Input list of names:
['Brandon Russell', 'Jerome Whitehead', 'Ryan Mack', 'Thomas Berry', 'Charles Tyler', 'Evelyn Christian', 'Steven Johnson', 'Aaron Graham', 'George Cook', 'Kyle Jones']

Presenter list after merging groups:
['Brandon Russell', 'Charles Tyler', 'Aaron Graham', 'Jerome Whitehead', 'Evelyn Christian', 'George Cook', 'Ryan Mack', 'Steven Johnson', 'Kyle Jones', 'Thomas Berry']


,name,group
0,Brandon Russell,group_0
1,Charles Tyler,group_1
2,Aaron Graham,group_2
3,Jerome Whitehead,group_0
4,Evelyn Christian,group_1
5,George Cook,group_2
6,Ryan Mack,group_0
7,Steven Johnson,group_1
8,Kyle Jones,group_2
9,Thomas Berry,group_0


In [6]:
# Let's drop a couple of names. Some names that we do not want on the schedule
np.random.seed(123)
drop_names = list(np.random.choice(presenter_list_0, size=2, replace=False))
print()
print(f'Dropping names: {drop_names}')
presenter_list_1 = [name for name in presenter_list_0 if name not in drop_names]
print(f'Presenter list after dropping the names:\n{presenter_list_1}')

# Still, we want to switch positions 1 and 4 in the sequence
presenter_list_2 = presenter_list_1.copy()
presenter_list_2[1], presenter_list_2[4] = presenter_list_2[4], presenter_list_2[1]
print()
print(f'Presenter list after switching positions 1 and 4:\n{presenter_list_2}')


Dropping names: ['Evelyn Christian', 'Brandon Russell']
Presenter list after dropping the names:
['Charles Tyler', 'Aaron Graham', 'Jerome Whitehead', 'George Cook', 'Ryan Mack', 'Steven Johnson', 'Kyle Jones', 'Thomas Berry']

Presenter list after switching positions 1 and 4:
['Charles Tyler', 'Ryan Mack', 'Jerome Whitehead', 'George Cook', 'Aaron Graham', 'Steven Johnson', 'Kyle Jones', 'Thomas Berry']


In [7]:
# Now, we are happy with that sequence and we want use it in the meeting schedule
presenter_list_3 = meet.create_name_sequence(name_sequence=presenter_list_2, merge_groups=False)
print()
print(f'Presenter sequence for meeting schedule:{presenter_list_3}')
print('Group distribution')
display(meet.name_df)


Presenter sequence for meeting schedule:['Charles Tyler', 'Ryan Mack', 'Jerome Whitehead', 'George Cook', 'Aaron Graham', 'Steven Johnson', 'Kyle Jones', 'Thomas Berry']
Group distribution


,name,group
0,Charles Tyler,group_1
1,Ryan Mack,group_0
2,Jerome Whitehead,group_0
3,George Cook,group_2
4,Aaron Graham,group_2
5,Steven Johnson,group_1
6,Kyle Jones,group_2
7,Thomas Berry,group_0


### Create the meeting schedule ###

In [8]:
# Create a meeting schedule for three months
start_date = '2024-08-07'
end_date = '2024-10-31'
# pick a start name
seed = 235
start_name = np.random.choice(meet.name_df['name'].unique(), size=1)[0]
print(start_name)
print()
cal = meet.create_timeboard(start_date=start_date,
                            end_date=end_date,
                            start_name=start_name)
display(cal)

Ryan Mack



,date,name,group,holiday,comment
0,2024-08-07,Ryan Mack,group_0,False,None
1,2024-08-14,Jerome Whitehead,group_0,False,None
2,2024-08-21,George Cook,group_2,False,None
3,2024-08-28,Aaron Graham,group_2,False,None
4,2024-09-04,Steven Johnson,group_1,False,None
5,2024-09-11,Kyle Jones,group_2,False,None
6,2024-09-18,Thomas Berry,group_0,False,None
7,2024-09-25,Charles Tyler,group_1,False,None
8,2024-10-02,Ryan Mack,group_0,False,None
9,2024-10-09,Jerome Whitehead,group_0,False,None


### Skip meeting dates with a comment ###

In [9]:
# We want to skip a date where we are going to have a party
party_date = '2024-09-18'

comment = 'Department party, no meeting'
name = 'Everyone'

cal_skipped = meet.skip_date(cal_df=cal, date=party_date, comment=comment, name='Everyone')

# As you can see, the skipped date is marked and the sequence continues after the skipped date
display(cal_skipped)

,date,name,group,holiday,comment
0,2024-08-07,Ryan Mack,group_0,False,None
1,2024-08-14,Jerome Whitehead,group_0,False,None
2,2024-08-21,George Cook,group_2,False,None
3,2024-08-28,Aaron Graham,group_2,False,None
4,2024-09-04,Steven Johnson,group_1,False,None
5,2024-09-11,Kyle Jones,group_2,False,None
6,2024-09-18,Everyone,group_0,False,"Department party, no meeting"
7,2024-09-25,Thomas Berry,group_0,False,None
8,2024-10-02,Charles Tyler,group_1,False,None
9,2024-10-09,Ryan Mack,group_0,False,None


In [10]:
# We cannot skip a date that is not on the list
# party_date = '2024-09-20'
# cal_skipped = meet.skip_date(cal_df=cal, date=party_date, comment=comment, name='Everyone')

### Swap two dates in meeting schedule ###

In [11]:
date_1 = '2024-08-21'
date_2 = '2024-10-02'

cal_swapped = meet.swap_dates(cal_df=cal_skipped, date_1=date_1, date_2=date_2)

display(cal_swapped)

,date,name,group,holiday,comment
0,2024-08-07,Ryan Mack,group_0,False,None
1,2024-08-14,Jerome Whitehead,group_0,False,None
2,2024-08-21,Charles Tyler,group_1,False,None
3,2024-08-28,Aaron Graham,group_2,False,None
4,2024-09-04,Steven Johnson,group_1,False,None
5,2024-09-11,Kyle Jones,group_2,False,None
6,2024-09-18,Everyone,group_0,False,"Department party, no meeting"
7,2024-09-25,Thomas Berry,group_0,False,None
8,2024-10-02,George Cook,group_2,False,None
9,2024-10-09,Ryan Mack,group_0,False,None


In [13]:
# Save the data
data_file = 'meetings.csv'
data_dir = os.path.join(os.environ.get('HOME'), 'data')
cal_swapped.to_csv(os.path.join(data_dir, data_file), index=False)

### Create a simple schedule with just a list of names ###

In [17]:
name_list = ['Andreas', 'Eva', 'Matthias', 'Manuela']
meeting = Meetings(name_list=name_list)
start_date = '2024-08-28'
end_date = '2024-12-31'
cal = meeting.create_timeboard(start_date=start_date, end_date=end_date, meeting_day=2)
display(cal)

,date,name,holiday,comment
0,2024-08-28,Andreas,False,None
1,2024-09-04,Eva,False,None
2,2024-09-11,Matthias,False,None
3,2024-09-18,Manuela,False,None
4,2024-09-25,Andreas,False,None
5,2024-10-02,Eva,False,None
6,2024-10-09,Matthias,False,None
7,2024-10-16,Manuela,False,None
8,2024-10-23,Andreas,False,None
9,2024-10-30,Eva,False,None
